# Assignment 3

## Part 1

### A

In [6]:
import mysql.connector
from datetime import datetime

conn = mysql.connector.connect(
    host=" Noahs-Mac-mini.local",
    user="root",
    password="sushiSQL",
    database="Assn3"
)
cursor = conn.cursor()

### B

In [8]:
create_table_query = """
CREATE TABLE IF NOT EXISTS `WikiEdit` (
    `RevisionID` INT PRIMARY KEY,
    `ArticleName` VARCHAR(500),
    `EditDate` DATE,
    `UserName` VARCHAR(50)
)
"""
cursor.execute(create_table_query)
print("complete")

### C

In [12]:
file_path = "data/wiki_edit.txt"
with open(file_path, 'r') as file:
    for line_number, line in enumerate(file, start=1):
        # Assuming the format is: RevisionID,ArticleName,EditDate,UserName
        fields = line.strip().split(',')

        if len(fields) == 4:
            try:
                revision_id, article_name, edit_date_str, user_name = fields

                # Convert EditDate to date object
                edit_date = datetime.strptime(edit_date_str, '%Y-%m-%d').date()

                # Insert data into the database
                insert_query = """
                INSERT INTO `WikiEdit` (`RevisionID`, `ArticleName`, `EditDate`, `UserName`)
                VALUES (%s, %s, %s, %s)
                """
                cursor.execute(insert_query, (int(revision_id), article_name, edit_date, user_name))
                conn.commit()
            except ValueError as e:
                print(f"Ignoring line {line_number}: {line.strip()} - {e}")
        else:
            print(f"Ignoring line {line_number}: {line.strip()} - Not enough values to unpack (expected 4, got {len(fields)})")

Ignoring line 1: 9019643 Anarchism 2005-01-01 ip:24.136.172.37 - Not enough values to unpack (expected 4, got 1)
Ignoring line 2: 9062018 Anarchism 2005-01-03 ip:137.53.22.23 - Not enough values to unpack (expected 4, got 1)
Ignoring line 3: 9085524 Anarchism 2005-01-03 ip:24.42.90.254 - Not enough values to unpack (expected 4, got 1)
Ignoring line 4: 9140836 Anarchism 2005-01-06 ip:69.61.184.201 - Not enough values to unpack (expected 4, got 1)
Ignoring line 5: 9160986 A 2005-01-06 Fvw - Not enough values to unpack (expected 4, got 1)
Ignoring line 6: 9243125 Achilles 2005-01-08 ip:24.3.52.22 - Not enough values to unpack (expected 4, got 1)
Ignoring line 7: 9034734 Abraham_Lincoln 2005-01-02 ip:24.85.3.38 - Not enough values to unpack (expected 4, got 1)
Ignoring line 8: 9082287 Abraham_Lincoln 2005-01-03 Polynova - Not enough values to unpack (expected 4, got 1)
Ignoring line 9: 9085580 Abraham_Lincoln 2005-01-03 ip:128.143.193.134 - Not enough values to unpack (expected 4, got 1)
I

### D

In [ ]:
largest_edit_query = """
SELECT ArticleName, COUNT(*) AS EditCount
FROM `WikiEdit`
GROUP BY ArticleName
ORDER BY EditCount DESC
LIMIT 1
"""
cursor.execute(largest_edit_query)
result = cursor.fetchone()
if result:
    article_name_with_most_edits = result[0]
    print(f"Article name with the largest number of edits: {article_name_with_most_edits}")

# Close the connection
cursor.close()
conn.close()